In [1]:
# !pip install transformers
# !pip install git+https://github.com/yuchenlin/LLM-Blender.git
# !pip install "pydantic<2"
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
# !pip install pip install accelerate

In [2]:
# !pip install pip install accelerate

In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM 
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AdamW, AutoTokenizer
from tqdm import tqdm
# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "beomi/polyglot-ko-12.8b-safetensors"  # safetensors 컨버팅된 레포
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

tokenizer = AutoTokenizer.from_pretrained(model_id,  eos_token='</s>')
model = AutoModelForCausalLM.from_pretrained(model_id)
                                             # quantization_config=bnb_config)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [5]:
# from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_int8_training
# # lora_config = LoraConfig(r=8,
# #                          lora_alpha=32,
# #                          target_modules=['q_proj','v_proj'],
# #                          lora_dropout=0.1,
# #                          # inference_mode=False,
# #                          bias="none",
# #                          task_type=TaskType.CAUSAL_LM)
# lora_config = LoraConfig(
#     r=8, 
#     lora_alpha=32, 
#     target_modules=["query_key_value"], 
#     lora_dropout=0.05, 
#     bias="none", 
#     task_type="CAUSAL_LM"
# )

# model.gradient_checkpointing_enable()

# model = prepare_model_for_int8_training(model)
# perf_model = get_peft_model(model, lora_config)



from peft import get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit, PeftModel, PeftConfig

peft_model = PeftModel.from_pretrained(
    model,    # 프롬프트 튜닝에 사용될 기본 모델
    "./alpha_lora_3_epoch/",     # Peft 모델이 저장된 위치
    is_trainable = False 
)

In [6]:
test_df = pd.read_csv("./data/test.csv")
# test_df
# test_df["질문"].iloc[107]

In [7]:

appedix_dict_1 = {}
for idx, row in test_df.iterrows():
    for q_col in ['질문','id']:
        if q_col == '질문':
            if row[q_col].count('?') >=2:
                appendix = row[q_col].split('?')[1] + "?"
                appendix = appendix.strip()
                row[q_col] = row[q_col].split('?')[0] + "?"
                row[q_col] = row[q_col].strip()
                appedix_dict_1[idx] = appendix
            else:
                row[q_col] = row[q_col].strip()
appedix_dict_1   

{0: '또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?',
 11: '또한, 부실 시공으로 인해 타공하자가 발생할 가능성이 있나요?',
 16: '그리고 유성페인트를 사용하는 것에 대한 부작용이 있을까요?',
 17: '그리고 반점이 생긴지 1년 이내인 하자에 대해 어떤 보수작업을 해야 하나요?',
 20: '또한, 피스 하자가 무엇인지 자세히 설명해 주실 수 있나요?',
 22: '또한, 겨울에 도배를 할 때 특별히 신경써야 할 사항이 있을까요?',
 23: '또한, 페인트가 남으면 어디에 보관하는 게 좋을까요?',
 24: '또한, 유광 자기질 타일은 얼마나 오랫동안 사용할 수 있을까요?',
 25: '그리고 스탠드조명을 선택할 때 주의할 점이 있을까요?',
 26: '그리고 불량 도배지를 사용할 경우 도배지가 얼마나 오랫동안 버틸까요?',
 27: '그리고 실크벽지의 교체 주기는 얼마나인가요?',
 28: '그리고 철골구조의 장점은 무엇인가요?',
 29: '그리고 조적식 구조란 무엇인가요?',
 33: '또한, 액체방수공사는 무엇을 하는 것인가요?',
 35: '그리고 장판이 남을 때 어떻게 처리해야 하나요?',
 38: '또한, 콘크리트 벽에 구멍을 뚫는 방법에는 어떤 도구나 기술을 사용해야 하나요?',
 39: '그리고 인테리어에 가장 많이 사용되는 도배재료는 무엇인가요?',
 42: '그리고 복도나 협소한 공간을 확장시키기 위해 가장 효과적인 방법이 무엇일까요?',
 48: '또한, 배관공사 시 통기구를 설치해야 하는 이유가 무엇인가요?',
 49: '또한, 규산질계 침투성 도포 방수공사는 어떤 방식으로 이루어지나요?',
 50: '또한, 내부와 외부 온도의 큰 차이로 인해 곰팡이 발생이 빨라지나요?',
 54: '그리고 기둥-보 구조 방식은 무엇을 의미하나요?',
 56: '그리고 새집증후군을 예방하는 데 가장 효과적인 방법은 무엇인가요?',
 59: '또한, 옥상 방수용 탄성 에멀전 페인트를 사용하는 장점은 무엇인가요?',
 63: 

In [8]:

appedix_dict_2 = {}
for idx, row in test_df.iterrows():
    for q_col in ['질문','id']:
        if q_col == '질문':
            if row[q_col].count('그리고') > 0 and row[q_col].count('이를') == 0:
                appendix = row[q_col].split('그리고')[1]
                appendix = appendix.strip()
                row[q_col] = row[q_col].split('그리고')[0]
                row[q_col] = row[q_col].strip()
                appedix_dict_2[idx] = appendix
            else:
                row[q_col] = row[q_col].strip()
appedix_dict_2     

{6: '합지벽지의 어떤 단점이 있나요?',
 15: '페인트 상도재의 역할과 사용 방법에 대해 알려주세요.',
 47: '습도가 높을 때 곰팡이가 어떻게 발생하는지 자세히 알고 싶습니다.',
 55: '도배지가 남으면 어떻게 처리해야 하나요?',
 64: '소화기 종류에는 어떤 것들이 있는지 알려주세요.',
 86: '토목이 무엇인지 설명해줘.',
 93: '셀룰로오스의 단점에 대해 설명해주세요.',
 94: '반려동물을 위한 바닥재에는 어떤 종류가 있는지 알려주세요.',
 98: '밀풀 사용 시 주의할 점은 무엇인가요?',
 99: '통나무구조 방식은 어떤 건물에 사용되는 건축 구조 방식인가요?',
 108: '펫테리어가 무엇인지 자세히 알려주세요.',
 119: '철근콘크리트 구조에 대해 좀 더 자세히 알려주세요.'}

In [9]:
appedix_dict_3 = {}
for idx, row in test_df.iterrows():
    for q_col in ['질문','id']:
        if q_col == '질문':
            if row[q_col].count('또한') > 0:
                appendix = row[q_col].split('또한')[1] + "또한"
                appendix = appendix.strip()
                row[q_col] = row[q_col].split('또한')[0] + "또한"
                row[q_col] = row[q_col].strip()
                appedix_dict_3[idx] = appendix
            else:
                row[q_col] = row[q_col].strip()

appedix_dict_3

{2: ', 압출법 단열판을 사용하는 것의 장점은 무엇인가요?또한',
 32: ', 세라믹 타일을 사용할 때 고려해야 할 단점은 무엇인가요?또한',
 45: ', 낡은 목재 가구의 흠집을 숨기는 방법을 알려주세요.또한',
 52: ', 속건형 유성 발수제의 사용 목적과 효과에 대해 알려주세요.또한',
 72: ', 강화마루의 장점은 무엇입니까?또한',
 89: ', 아이소핑크의 장점은 무엇인가요?또한'}

In [10]:
peft_model.to(device)
peft_model.eval()

OutOfMemoryError: CUDA out of memory. Tried to allocate 300.00 MiB (GPU 0; 47.54 GiB total capacity; 46.50 GiB already allocated; 242.25 MiB free; 46.51 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [20]:

# Inference를 위한 test.csv 파일 로드
# test = pd.read_csv('./data/test.csv')

# test.csv의 '질문'에 대한 '답변'을 저장할 리스트
file_path = "result_alpha.txt"
total_preds = []
# '질문' 컬럼의 각 질문에 대해 답변 생성
# for idx,model in enumerate(model_list):
preds = []
idx = 0
for test_question in tqdm(test_df['질문']):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    with torch.no_grad():

        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.5,
        #     top_k=120,
        #     top_p=1.5,
        #     repetition_penalty=1.0,
        #     do_sample=True,
        #     num_beams=120,
        #     # no_repeat_ngram_size=3,
        #     # num_return_sequences=1,
        #     # eos_token_id=tokenizer.eos_token_id
        # )
        output_sequences = model.generate(
            input_ids=input_ids.to(device),
            max_length=800,
            # min_length=20,
            # temperature=1.2,
            # top_k=500,
            # top_p=1.2,
            # repetition_penalty=1.5,
            # do_sample=True,
            # num_beams=18,
            # no_repeat_ngram_size=5,
            # num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )

    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        print(full_text)
        answer_start = full_text.find(tokenizer.eos_token) 
        anser_end = full_text.rfind(tokenizer.eos_token) 
        answer_only = full_text[answer_start+4:anser_end].strip()
        answer_only = answer_only.replace('\n', ' ')
        print(answer_only)
        preds.append(answer_only)
        answer_only = answer_only+'\n'
    if idx==0:
        with open(file_path, "w") as file:
            file.write(answer_only)
    else:
        with open(file_path, "a") as file:
            file.write(answer_only)
    idx+=1


    # with open(csv_file_path, mode='w') as csv_file:
    #     csv_writer = csv.writer(csv_file)
    #     csv_writer.writerow(answer_only)
# break
# total_preds.append(preds)
# model = model.cpu()
# torch.cuda.empty_cache()


  0%|                                                  | 0/130 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:30003 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  0%|                                                  | 0/130 [00:01<?, ?it/s]


RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

In [11]:
file_path = "result_alpha_appendix_1_r2.txt"


for idx, test_question in tqdm(appedix_dict_1.items()):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    with torch.no_grad():

        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.5,
        #     top_k=120,
        #     top_p=1.5,
        #     repetition_penalty=1.0,
        #     do_sample=True,
        #     num_beams=120,
        #     # no_repeat_ngram_size=3,
        #     # num_return_sequences=1,
        #     # eos_token_id=tokenizer.eos_token_id
        # )
        output_sequences = model.generate(
            input_ids=input_ids.to(device),
            max_length=800,
            min_length=20,
            temperature=0.8,
            top_k=500,
            top_p=0.8,
            repetition_penalty=1.5,
            do_sample=True,
            num_beams=18,
            no_repeat_ngram_size=5,
            # num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )


    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        print(full_text)
        answer_start = full_text.find(tokenizer.eos_token) 
        anser_end = full_text.rfind(tokenizer.eos_token) 
        answer_only = full_text[answer_start+4:anser_end].strip()
        answer_only = answer_only.replace('\n', ' ')
        preds.append(answer_only)
        print(answer_only)
        answer_only = str(idx)+answer_only+'\n'
    if idx==0:
        with open(file_path, "w") as file:
            file.write(answer_only)
    else:
        with open(file_path, "a") as file:
            file.write(answer_only)
    idx+=1

  0%|                                                                                                                                                                                    | 0/46 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:30003 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  2%|███▋                                                                                                                                                                        | 1/46 [00:03<02:54,  3.88s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain

원목사이딩의 단점은 가격대가 높을 뿐 아니라 관리가 어렵고 습기에 민감하여 뒤틀림, 부서짐, 수축/팽창이 발생할 수 있다는 점입니다. 또한 특히 곰팡이와 곤충에 노출될 경우 훼손될 가능성이 높다는 점도 감안해야 합니다.


  4%|███████▍                                                                                                                                                                    | 2/46 [00:11<04:17,  5.84s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:30003 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


타공불량은 부실한 시공으로 인해 발생할 수 있습니다. 예를 들어, 적절하지 않은 도구로 작업하거나 타공 부위의 크기를 정확히 측정하지 않고 작업하면 타공불량이 발생할 수 있으며, 이는 부실한 시공으로 이어질 수 있습니다. 따라서 시공자가 부실한 시공을 했거나 잘못된 도구로 작업하는 것을 방지하기 위해 제조사의 권장사항을 준수하고, 적절한 도구와 기술을 사용하여 작업하는 것이 중요합니다.


  7%|███████████▏                                                                                                                                                                | 3/46 [00:16<04:11,  5.86s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:30003 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

내진구조는 건물의 구조물에 충격이 발생했을 때 붕괴되지 않도록 설계된 구조를 말합니다. 내진구조는 지진 발생 시 건물의 붕괴나 큰 피해를 최소화하기 위해 설계된 시스템으로, 건물의 기초나 건물 구조물에 충격을 완화시키는 장치 등을 설치하여 지진으로 인한 피해를 최소화합니다.


 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎   | 45/46 [04:03<00:03,  3.31s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:30003 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


아파트 도배평수는 일반적으로 분양 평수*2.5로 계산합니다.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46/46 [04:08<00:00,  5.40s/it]

오리지널징크는 다른 징크에 비해 수명이 길고 다양한 패턴과 디자인이 가능하며 친환경적이고 금속 부식에 대한 내식성이 뛰어나 유지보수가 쉽다는 장점이 있습니다. 또한 오리지널징크는 가벼우면서도 매우 견고하며 낮은 유지보수 비용과 높은 재활용 가능성을 갖고 있습니다.


In [12]:
file_path = "result_alpha_appendix_2_r2.txt"


for idx, test_question in tqdm(appedix_dict_2.items()):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    with torch.no_grad():

        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.5,
        #     top_k=120,
        #     top_p=1.5,
        #     repetition_penalty=1.0,
        #     do_sample=True,
        #     num_beams=120,
        #     # no_repeat_ngram_size=3,
        #     # num_return_sequences=1,
        #     # eos_token_id=tokenizer.eos_token_id
        # )
        output_sequences = model.generate(
            input_ids=input_ids.to(device),
            max_length=800,
            min_length=20,
            temperature=1.2,
            top_k=500,
            top_p=1.2,
            repetition_penalty=1.5,
            do_sample=True,
            num_beams=18,
            no_repeat_ngram_size=5,
            # num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )


    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        print(full_text)
        answer_start = full_text.find(tokenizer.eos_token) 
        anser_end = full_text.rfind(tokenizer.eos_token) 
        answer_only = full_text[answer_start+4:anser_end].strip()
        answer_only = answer_only.replace('\n', ' ')
        preds.append(answer_only)
        print(answer_only)
        print(answer_only)
        answer_only = str(idx)+answer_only+'\n'
    if idx==0:
        with open(file_path, "w") as file:
            file.write(answer_only)
    else:
        with open(file_path, "a") as file:
            file.write(answer_only)
    idx+=1

  0%|                                                                                                                                                                                    | 0/12 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:30003 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  8%|██████████████▎                                                                                                                                                             | 1/12 [00:01<00:21,  1.96s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain

합지벽지는 가격이 저렴하고 이음새 겹침으로 시공이 쉬우며 실크벽지보다 친환경인 점이 장점입니다.
합지벽지는 가격이 저렴하고 이음새 겹침으로 시공이 쉬우며 실크벽지보다 친환경인 점이 장점입니다.


 17%|████████████████████████████▋                                                                                                                                               | 2/12 [00:07<00:39,  3.95s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:30003 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


페인트 상도기는 최종 마감으로 칠하는 페인트로, 영어로는 Top Coating이라고 합니다. 상도재는 하도재와 주도재의 제품을 보호하는 역할을 담당하며, 외부 표면에 노출될 때의 열 및 기후에 강하며 내구성이 높은 특징을 가지고 있습니다. 이러한 장점 때문에 가격이 다소 비쌉니다.
페인트 상도기는 최종 마감으로 칠하는 페인트로, 영어로는 Top Coating이라고 합니다. 상도재는 하도재와 주도재의 제품을 보호하는 역할을 담당하며, 외부 표면에 노출될 때의 열 및 기후에 강하며 내구성이 높은 특징을 가지고 있습니다. 이러한 장점 때문에 가격이 다소 비쌉니다.


 25%|███████████████████████████████████████████                                                                                                                                 | 3/12 [00:08<00:26,  2.90s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:30003 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

통나무구조 방식이란 주로 원형이나 각형의 내력벽으로 구성되며, 나머지 바닥이나 지붕은 2x4 경량 목구조와 유사한 방식으로 조립됩니다. 이 구조는 벽체가 통나무를 쌓는 형태이기 때문에 1층 높이에서 7~12cm 정도의 침하가 장기적으로 발생할 수 있으므로, 창문틀 등의 개구부에는 침하를 고려한 디테일을 고려해야 합니다. 통나무구조 방식은 자연적이고 따뜻한 분위기를 연출해내기 때문에 주택 및 휴양시설에서 선호되는 구조 방식 중 하나입니다.
통나무구조 방식이란 주로 원형이나 각형의 내력벽으로 구성되며, 나머지 바닥이나 지붕은 2x4 경량 목구조와 유사한 방식으로 조립됩니다. 이 구조는 벽체가 통나무를 쌓는 형태이기 때문에 1층 높이에서 7~12cm 정도의 침하가 장기적으로 발생할 수 있으므로, 창문틀 등의 개구부에는 침하를 고려한 디테일을 고려해야 합니다. 통나무구조 방식은 자연적이고 따뜻한 분위기를 연출해내기 때문에 주택 및 휴양시설에서 선호되는 구조 방식 중 하나입니다.


 92%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊              | 11/12 [00:50<00:05,  5.80s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:30003 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


펫테리어란 ‘반려동물(pet)’과 ‘인테리어(interior)’의 합성어로, 반려동물과의 생활을 고려한 편의성을 높이면서도 주거 공간의 인테리어를 고려하는 스타일을 말합니다. 즉, 반려동물과 관련된 편의시설과 인테리어 소품을 활용하여 쾌적하고 안락한 공간을 조성하는 것이 펫테리어의 핵심입니다. 이를 통해 주거 환경을 반려동물과 함께하는 가족들에게 쾌적한 공간을 제공하는 것이 목표입니다.
펫테리어란 ‘반려동물(pet)’과 ‘인테리어(interior)’의 합성어로, 반려동물과의 생활을 고려한 편의성을 높이면서도 주거 공간의 인테리어를 고려하는 스타일을 말합니다. 즉, 반려동물과 관련된 편의시설과 인테리어 소품을 활용하여 쾌적하고 안락한 공간을 조성하는 것이 펫테리어의 핵심입니다. 이를 통해 주거 환경을 반려동물과 함께하는 가족들에게 쾌적한 공간을 제공하는 것이 목표입니다.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:56<00:00,  4.75s/it]

철근콘크리트구조는 콘크리트의 압축 강도를 활용하면서도 인장 응력에 대한 취약점을 보완하기 위해 철근을 사용하는 구조입니다. 일반적으로 도심 지역에서 많이 사용되며, 내구성, 내화 및 내진 성능이 뛰어나지만, 철근의 보강으로 인해 구조물의 중량이 증가하여 고층 건물에서는 아래층의 보나 기둥 단면적이 커져야 하는 문제가 있습니다.
철근콘크리트구조는 콘크리트의 압축 강도를 활용하면서도 인장 응력에 대한 취약점을 보완하기 위해 철근을 사용하는 구조입니다. 일반적으로 도심 지역에서 많이 사용되며, 내구성, 내화 및 내진 성능이 뛰어나지만, 철근의 보강으로 인해 구조물의 중량이 증가하여 고층 건물에서는 아래층의 보나 기둥 단면적이 커져야 하는 문제가 있습니다.


In [13]:
file_path = "result_alpha_appendix_3_r2.txt"


for idx, test_question in tqdm(appedix_dict_3.items()):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    
    input_ids = tokenizer.encode(test_question + tokenizer.eos_token, return_tensors='pt')

    # 답변 생성
    with torch.no_grad():

        # output_sequences = model.generate(
        #     input_ids=input_ids.to(device),
        #     max_length=2000,
        #     min_length=20,
        #     temperature=1.5,
        #     top_k=120,
        #     top_p=1.5,
        #     repetition_penalty=1.0,
        #     do_sample=True,
        #     num_beams=120,
        #     # no_repeat_ngram_size=3,
        #     # num_return_sequences=1,
        #     # eos_token_id=tokenizer.eos_token_id
        # )
        output_sequences = model.generate(
            input_ids=input_ids.to(device),
            max_length=800,
            min_length=20,
            temperature=1.2,
            top_k=500,
            top_p=1.2,
            repetition_penalty=1.5,
            do_sample=True,
            num_beams=18,
            no_repeat_ngram_size=5,
            # num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id
        )


    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        print(full_text)
        answer_start = full_text.find(tokenizer.eos_token) 
        anser_end = full_text.rfind(tokenizer.eos_token) 
        answer_only = full_text[answer_start+4:anser_end].strip()
        answer_only = answer_only.replace('\n', ' ')
        preds.append(answer_only)
        print(answer_only)
        answer_only = str(idx)+answer_only+'\n'
    if idx==0:
        with open(file_path, "w") as file:
            file.write(answer_only)
    else:
        with open(file_path, "a") as file:
            file.write(answer_only)
    idx+=1

  0%|                                                                                                                                                                                     | 0/6 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:30003 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
 17%|████████████████████████████▊                                                                                                                                                | 1/6 [00:05<00:25,  5.08s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain

압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있는데, 이는 건강한 실내 환경을 유지하는 데 도움을 줍니다. 또한, 열전도율이 가장 낮아서 효율적인 단열 효과를 제공하며 시공이 간편하여 공사 기간을 단축하는 장점이 있습니다.


 33%|█████████████████████████████████████████████████████████▋                                                                                                                   | 2/6 [00:10<00:20,  5.16s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:30003 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


세라믹 타일의 단점으로는 시공이 어려워 하자가 발생할 확률이 높고, 숙련된 기술자가 시공을 맡아야 하므로 가격과 시공비가 상대적으로 높다는 점이 있습니다. 또한 일반 타일보다 슬립리스 처리가 부족하여 미끄러질 수 있는 위험이 있을 수 있습니다.


 50%|██████████████████████████████████████████████████████████████████████████████████████▌                                                                                      | 3/6 [00:15<00:15,  5.13s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:30003 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

속건형 유성 발수제로 도장된 표면에 도장된 색상이 변하지 않도록 하는 방법이 있을까요?


 83%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                            | 5/6 [00:22<00:04,  4.42s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:30003 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


강화마루는 원목 무늬 필름지가 하드코팅 처리된 바닥재를 의미합니다. 일반적으로 평당 7~12 만원의 가격대를 가지고 있습니다. 이 바닥재는 온도나 습도에 의한 변색이 거의 없고, 강도가 뛰어나 수명이 긴 장점이 있습니다. 하지만, 필름지로 코팅처리되어 있기 때문에 질감이나 보행감이 조금 떨어진다는 단점이 있습니다.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:30<00:00,  5.12s/it]

아이소핑크의 단점은 주로 비용 측면에서 언급됩니다. 이 외에도 아이소핑크는 단열재 중에서 열전도율이 가장 낮아 에너지 절약 효과가 크며, 습기에 강하고 곰팡이 및 세균 증식을 막아주는 특징이 있습니다. 또한 시공이 간편하여 공사 기간과 비용을 절약할 수 있는 장점이 있습니다.


In [14]:


# source_prefix = "<|source|>"
# cand1_prefix = "<|candidate1|>"
# cand2_prefix = "<|candidate2|>"
# inputs = ["hello!", "I love you!"]
# candidates_A = ["hi!", "I hate you!"]
# candidates_B = ["f**k off!", "I love you, too!"]
# def tokenize_pair(sources:List[str], candidate1s:List[str], candidate2s:List[str], source_max_length=1224, candidate_max_length=412):
#     ids = []
#     assert len(sources) == len(candidate1s) == len(candidate2s)
#     max_length = source_max_length + 2 * candidate_max_length
#     for i in range(len(sources)):
#         source_ids = tokenizer.encode(source_prefix + sources[i], max_length=source_max_length, truncation=True)
#         candidate_max_length = (max_length - len(source_ids)) // 2
#         candidate1_ids = tokenizer.encode(cand1_prefix + candidate1s[i], max_length=candidate_max_length, truncation=True)
#         candidate2_ids = tokenizer.encode(cand2_prefix + candidate2s[i], max_length=candidate_max_length, truncation=True)
#         ids.append(source_ids + candidate1_ids + candidate2_ids)
#     encodings = tokenizer.pad({"input_ids": ids}, return_tensors="pt", padding="max_length", max_length=max_length)
#     return encodings

# encodings = tokenize_pair(inputs, candidates_A, candidates_B)
# encodings = {k:v.to(pairrm.device) for k,v in encodings.items()}
# outputs = pairrm(**encodings)
# logits = outputs.logits.tolist()
# comparison_results = outputs.logits > 0
# print(logits)
# # [1.9003021717071533, -1.2547134160995483]
# print(comparison_results)